In [1]:
# Setup the google colab
!pip install xmnlp

     |████████████████████████████████| 23.7MB 137kB/s 
  Created wheel for xmnlp: filename=xmnlp-0.2.3-cp36-none-any.whl size=23707964 sha256=7b25bebf8dd03dcaac6b3a1aad0df0a64b29a97eda7d8e87662c21a191b599ef
  Stored in directory: /root/.cache/pip/wheels/1d/05/70/4a9b15884cdd6997fbc006fd6a9c2b3f5ca66857b6ac37100c
Successfully built xmnlp


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
isColab = True

googlePath = "./drive/MyDrive/L101Project/" if isColab else "./"
cpuPools = 2 if isColab else 8

In [4]:
import tensorflow as tf
from tensorflow import keras
physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import Session

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = Session(config=config)
# tf.compat.v1.disable_eager_execution()


In [6]:
import csv
import numpy as np
import xmnlp
import itertools

import collections
import operator
from functools import reduce
import json
from multiprocessing import Pool
from keras.preprocessing.sequence import pad_sequences

In [7]:
# Load NER dataset

In [15]:
import pandas as pd
train_ner_path =  "./ner_train.txt"
train = pd.read_table(train_ner_path, header=None, names=['token', 'label'])  # don't drop the empty lines yet, they show up as NaN in the data frame
train.head(n=50)

,token,label
0,海,O
1,钓,O
2,比,O
3,赛,O
4,地,O
5,点,O
6,在,O
7,厦,B-LOC
8,门,I-LOC
9,与,O


In [16]:
set(train["label"])

{'B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O', nan}

In [18]:
import numpy as np

# in order to convert word tokens to integers: list the set of token types
token_vocab = train.token.unique().tolist()
oov = len(token_vocab)  # OOV (out of vocabulary) token as vocab length (because that's max.index + 1)

# convert word tokens to integers
def token_index(tok):
  ind = tok
  if not pd.isnull(tok):  # new since last time: deal with the empty lines which we didn't drop yet
    if tok in token_vocab:  # if token in vocabulary
      ind = token_vocab.index(tok)
    else:  # else it's OOV
      ind = oov
  return ind

# training labels: convert BIO to integers
def bio_index(bio):
  ind = bio
  if not pd.isnull(bio):  # deal with empty lines
    if bio=='B-LOC':
      ind = 0
    elif bio=='B-ORG':
      ind = 1
    elif bio=='B-PER':
      ind = 2
    elif bio=='I-LOC':
      ind = 3
    elif bio=='I-ORG':
      ind = 4
    elif bio=='I-PER':
      ind = 5
    elif bio=='O':
      ind = 6
  return ind

# pass a data frame through our feature extractor
def extract_features(txt_orig,istest=False):
  txt = txt_orig.copy()
  tokinds = [token_index(u) for u in txt['token']]
  txt['token_indices'] = tokinds
  if not istest:  # can't do this with the test set
    bioints = [bio_index(b) for b in txt['label']]
    txt['bio'] = bioints
  return txt

train_copy = extract_features(train)
train_copy.head(n=30)

,token,label,token_indices,bio
0,海,O,0.0,6.0
1,钓,O,1.0,6.0
2,比,O,2.0,6.0
3,赛,O,3.0,6.0
4,地,O,4.0,6.0
5,点,O,5.0,6.0
6,在,O,6.0,6.0
7,厦,B-LOC,7.0,0.0
8,门,I-LOC,8.0,3.0
9,与,O,9.0,6.0


In [19]:
def tokens2sequences(txt_orig,istest=False):
  '''
  Takes panda dataframe as input, copies, and adds a sequence index based on full-stops.
  Outputs a dataframe with sequences of tokens, named entity labels, and token indices as lists.
  '''
  txt = txt_orig.copy()
  txt['sequence_num'] = 0
  seqcount = 0
  for i in txt.index:  # in each row...
    txt.loc[i,'sequence_num'] = seqcount  # set the sequence number
    if pd.isnull(txt.loc[i,'token']):  # increment sequence counter at empty lines
      seqcount += 1
  # now drop the empty lines, group by sequence number and output df of sequence lists
  txt = txt.dropna()
  if istest:  # test set doesn't have labels
    txt_seqs = txt.groupby(['sequence_num'],as_index=False)[['token', 'token_indices', "upos"]].agg(lambda x: list(x))
  else:
    txt_seqs = txt.groupby(['sequence_num'],as_index=False)[['token', 'bio', 'token_indices', "label"]].agg(lambda x: list(x))
  return txt_seqs

print("This cell takes a little while to run: be patient :)")
train_seqs = tokens2sequences(train_copy)
train_seqs.head()

This cell takes a little while to run: be patient :)


KeyError: ignored

In [ ]:
# Chinese word segamentation

In [ ]:
pkuTrain = googlePath + "./corpus/cws/icwb2-data/training/pku_training.utf8"
pkuTest = googlePath + "./corpus/cws/icwb2-data/testing/pku_test.utf8"

f = open(pkuTest, "r")
temp = f.readlines()
con = 0
for item in temp:
    con+=1
    if(item == "\n"):
        print(con)

In [ ]:
## Create Corpus for Chinese Word Segamentation

In [ ]:
import pandas as pd
pkuTrain = googlePath + "./corpus/cws/icwb2-data/training/pku_training.utf8"
pkuTest = googlePath + "./corpus/cws/icwb2-data/testing/pku_test.utf8"
train = pd.read_table(pkuTrain,  encoding='utf8', header=None, names=['input'])  # don't drop the empty lines yet, they show up as NaN in the data frame
# train.head(n=1)


In [ ]:
pkuTest = googlePath  + "./corpus/cws/icwb2-data/testing/pku_test.utf8"
test = pd.read_table(pkuTest,  encoding='utf8', header=None, names=['input']) 
test["raws"] = test["input"]
# test.head(n=1)

In [ ]:
def prepross_data(train_Data):
    inputs = train_Data["input"][:]
    print(len(inputs))
    raws = []
    tokenList = []
    
    for item in inputs:
        item = item.replace("   ", "  ")

        raw = item.replace(" ", "")
#         print(item)
#         print(raw)
        raws.append(raw)
        tokens = (item.split("  ")[:-1])
        tokenList.append(tokens)
#         print(tokens)
    train_Data["raws"] = raws
    train_Data["tokenList"] = tokenList
    dictionary_train_word = list(set(reduce(operator.add,train_Data["tokenList"])))
    dictionary_train_char =  list(set(reduce(operator.add,train_Data["raws"])))    
    return train_Data, dictionary_train_char, dictionary_train_word
train, train_dic_char, train_dic_word  = prepross_data(train)

19054


In [ ]:
# Get redical dictionary

# Get redical of a chinese character:
def str_get_redical(st):
    return xmnlp.radical(st)

def get_redical_dic_from_char_dic(char_dic):
    return list(set(str_get_redical("".join(char_dic))))

train_dic_redical = get_redical_dic_from_char_dic(train_dic_char)
print(len(train_dic_redical))

(Lazy Load) Loading model...
235


In [ ]:
# Build up subcharacter dictionary
# Build up Chinese sub-character unites diactionary.
chaiZi_Dic = {}
with open(googlePath + "corpus/chaizi/chaizi-jt.txt", 'r') as f:
    reader = csv.reader(f,delimiter='\t')
    for row in reader:
        chaiZi_Dic[row[0]] = row[1]
        
def getSubChar(charCN):
    if(charCN in chaiZi_Dic.keys()):
        return chaiZi_Dic[charCN].split(" ")
    elif charCN in train_dic_char:
        return [charCN]
    else:
        return [None]

In [ ]:
def str_get_subchar(st):
#     for item in st:
#         print(item)
#         print(getSubChar(item))
    return [getSubChar(item) for item in st]

def get_subchar_dic_from_char_dic(char_dic):
    return list(set(itertools.chain.from_iterable(str_get_subchar("".join(char_dic)))))

train_dic_subchar = get_subchar_dic_from_char_dic(train_dic_char)
print(len(train_dic_subchar))

1328


In [ ]:
# Validate the pre-processing
def validate_data(data):
    tkList = data["tokenList"]
    raws = data["raws"]
    for (tokens, raw) in zip(tkList,raws):
#         print(tokens)
        
#         print(raw)
        temp  = "".join(tokens)
#         print(temp)
        assert(temp == raw)
        if not (temp == raw):
            print(temp)
            print(raw)
            print(tokens)
validate_data(train)

In [ ]:
# print(len(train_dic_char))
# print(len(set(train_dic_char)))
# print(len(train_dic_word))
# print(len(set(train_dic_word)))
# with open(googlePath + "./corpus/cws/icwb2-data/gold/pku_training_words.utf8", 'r') as f:
#     content = f.readlines()  
#     print(len(content))

In [ ]:
train['raws'][1]

'中共中央总书记、国家主席江泽民'

In [ ]:
# Feature extraction

oov = len(train_dic_char)
print(oov)

oov_redical = len(train_dic_redical)
print(oov_redical)

oov_subchar = len(train_dic_subchar)
print(oov_subchar)

def token_index(tok):
    ind = tok
    if tok in train_dic_char:  # if token in vocabulary
        ind = train_dic_char.index(tok)
    else:  # else it's OOV
        ind = oov
    return ind

def str_token_index(string):
    return [token_index(x) for x in (string)]




def redical_index(red):
    ind = oov_redical
    if red in train_dic_redical:  # if token in vocabulary
        ind = train_dic_redical.index(red)
    return ind

def str_red_index(string):
    return [redical_index(x) for x in str_get_redical(string)]


def subchar_index(subchar):
    ind = oov_subchar
    if subchar in train_dic_subchar:
        ind = train_dic_subchar.index(subchar)
    return ind

def str_subchar_index(string):
    return [[subchar_index(temp) for temp in x] for x in str_get_subchar(string)]


# Get Begin Middle End Single sequence
# B 0 M 1 E 2 S 3

def str_bmes_idx(tokenList):
    answer = []
    for item in tokenList:
        if len(item) == 0:
            raise NameErro("Zero Length Word")
        if len(item) == 1:
            answer.append(3)
        else:
            answer.append(0)
            for item in range(len(item) - 2):
                answer.append(1)
            answer.append(2)
    return answer

def extract_features(data_set, isTest=False):
    data_temp = data_set.copy()

    # Idx for chars
    with Pool(8) as p:
        tokinds = p.map(str_token_index,data_temp['raws'])
#     tokinds = [list(map(token_index, u)) for u in data_temp['raws']]

    with Pool(8) as p:
        redinds = p.map(str_red_index,data_temp['raws'])
        
    with Pool(8) as p:
        subcharidx = p.map(str_subchar_index,data_temp['raws'])
        

    data_temp["tokenIdx"] = tokinds
    data_temp["redIdx"] = redinds
    data_temp["subcharIdx"] = subcharidx
    
    # BIO
    if(not isTest):
        data_temp["bmes"] = [str_bmes_idx(u) for u in data_temp['tokenList']]
        assert (list(map(len,data_temp["bmes"])) == list(map(len,data_temp["tokenIdx"])))

        
#     print(data_temp["bmes"])
#    assert reduce(operator.and_, list(map(len,data_temp["bmes"])) == list(map(len,data_temp["tokenIdx"])))
    
    # Get redical
    

    
    
    
    
    return data_temp
#     txt['token_indices'] = tokinds
#     if not istest:  # can't do this with the test set
#         bioints = [bio_index(b) for b in txt['bio_only']]
#         txt['bio_only'] = bioints
#     return txt

4698
235
1328


In [ ]:
str_red_index("花丸晴琉")

[182, 207, 60, 90]

In [ ]:
%%time
# MultiThreading
train_feature = extract_features(train)
# train_feature.head(5)

CPU times: user 4.48 s, sys: 523 ms, total: 5 s
Wall time: 3min 48s


In [ ]:
train_feature.head(1)

,input,raws,tokenList,tokenIdx,redIdx,subcharIdx,bmes
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...,迈向充满希望的新世纪——一九九八年新年讲话（附图片１张）,"[迈向, 充满, 希望, 的, 新, 世纪, ——, 一九九八年, 新年, 讲话, （, 附...","[2775, 4641, 873, 1797, 1428, 2458, 118, 1570,...","[159, 208, 105, 170, 225, 49, 53, 92, 222, 98,...","[[410, 1149], [1279, 116, 1291], [932, 524], [...","[0, 2, 0, 2, 0, 2, 3, 3, 0, 2, 0, 2, 0, 1, 1, ..."


In [ ]:
def find_longest_sequence(data_with_features):
#     assert (np.max(list(map(len, data_with_features["tokenIdx"])))) == (np.max(list(map(len, data_with_features["bmes"]))))
    return (np.max(list(map(len, data_with_features["tokenIdx"]))))
    
    

In [ ]:
%%time
temp = extract_features(test, isTest=True)
test_feature = temp
# test_feature.head(5)

CPU times: user 215 ms, sys: 255 ms, total: 471 ms
Wall time: 21.9 s


In [ ]:
test_feature.head(5)

,input,raws,tokenIdx,redIdx,subcharIdx
0,共同创造美好的新世纪——二○○一年新年贺词,共同创造美好的新世纪——二○○一年新年贺词,"[678, 2926, 997, 63, 4686, 1681, 118, 1570, 37...","[185, 208, 160, 159, 47, 200, 53, 92, 222, 98,...","[[8, 285], [116, 651, 1291], [39, 959], [410, ..."
1,（二○○○年十二月三十一日）（附图片1张）,（二○○○年十二月三十一日）（附图片1张）,"[1421, 1514, 4380, 4380, 4380, 1110, 3999, 151...","[28, 76, 28, 28, 28, 196, 184, 76, 49, 222, 18...","[[352], [651, 651], [989], [989], [989], [971,..."
2,女士们，先生们，同志们，朋友们：,女士们，先生们，同志们，朋友们：,"[4366, 1178, 3189, 50, 2068, 3562, 3189, 50, 2...","[200, 42, 103, 28, 105, 83, 103, 28, 208, 67, ...","[[108, 651, 971], [608, 651], [533, 41], [68],..."
3,2001年新年钟声即将敲响。人类社会前进的航船就要驶入21世纪的新航程。中国人民进入了向现代...,2001年新年钟声即将敲响。人类社会前进的航船就要驶入21世纪的新航程。中国人民进入了向现代...,"[1397, 4424, 4424, 4447, 1110, 1570, 1110, 205...","[28, 28, 28, 28, 196, 92, 196, 54, 42, 8, 113,...","[[328], [1035], [1035], [1058], [971, 876, 127..."
4,在这个激动人心的时刻，我很高兴通过中国国际广播电台、中央人民广播电台和中央电视台，向全国各族...,在这个激动人心的时刻，我很高兴通过中国国际广播电台、中央人民广播电台和中央电视台，向全国各族...,"[197, 613, 1553, 2914, 3055, 3941, 2352, 118, ...","[23, 159, 193, 170, 118, 133, 67, 53, 60, 160,...","[[651, 533, 693], [410, 506], [533, 297], [947..."


In [ ]:

train_longest = find_longest_sequence(train_feature)
print(train_longest)

test_longest = find_longest_sequence(test_feature)
print(test_longest)

seq_longest = np.max([train_longest,test_longest])
print(seq_longest)


1019
626
1019


In [ ]:
# find out max sub char length 



In [ ]:
subchar_seq_length = np.max(list(map(lambda x : len(getSubChar(x)),train_dic_char)))
subchar_seq_length = 5
subsubchar_padtok = oov_subchar + 1
seq_length = seq_longest

def padd_char(seq):
    temp_char_seqs_padded = []
    for item in seq["subcharIdx"]:
        temp_pad = pad_sequences(item, maxlen=subchar_seq_length,
                                  dtype='int32', padding='post', truncating='post', value=subsubchar_padtok)
  
        a = temp_pad
        b = [[subsubchar_padtok for i in range(subchar_seq_length)] for _ in range(0, seq_length - len(temp_pad))]
        if len(b) == 0:
            c = a
        else:
            c = np.concatenate((a, b))
        
    # print(len(c))
        temp_char_seqs_padded.append(c)
  # print(len(temp_char_seqs_padded))
    return temp_char_seqs_padded

In [ ]:
train_subchar_seqs_padded = padd_char(train_feature)


In [ ]:
seq_length = seq_longest

# a new dummy token index, one more than OOV
padtok = oov+1
red_padtok = oov_redical + 1
print('The padding token index is %i' % padtok)

# use pad_sequences, padding or truncating at the end of the sequence (default is 'pre')
train_seqs_padded = pad_sequences(train_feature['tokenIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=padtok)
print('Example of padded token sequence:')
print(train_seqs_padded[1])


train_red_padded = pad_sequences(train_feature['redIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=red_padtok)


train_subchar_seqs_padded = padd_char(train_feature)





# Prepare Test set.
test_seqs_padded = pad_sequences(test_feature['tokenIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=padtok)

test_rad_padded = pad_sequences(test_feature['redIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=red_padtok)

test_subchar_seqs_padded = padd_char(test_feature)


The padding token index is 4699
Example of padded token sequence:
[4192  678 4192 ... 4699 4699 4699]


In [ ]:
from keras.utils import to_categorical

# get lists of named entity labels, padded with a null label (=3)
padlab = 4
train_labs_padded = pad_sequences(train_feature['bmes'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=padlab)

# convert those labels to one-hot encoding
n_labs = 5
train_labs_onehot = [to_categorical(i, num_classes=n_labs) for i in train_labs_padded]

# # follow the print outputs below to see how the labels are transformed
# print('Length of input sequence: %i' % len(train_labs_padded[1]))
# print('Length of label sequence: %i' % len(train_labs_onehot[1]))
# print(train_labs_padded[1][:11])
# print(train_labs_onehot[1][:11])

In [ ]:
# load Keras and TensorFlow



# our final vocab size is the padding token + 1 (OR length of vocab + OOV + PAD)
vocab_size = padtok+1
red_size = red_padtok+1
subchar_size = subsubchar_padtok + 1

print(vocab_size==len(train_dic_char)+2)
embed_size = 128 # y an embedding size of 128 (could tune this)

# list of metrics to use: true & false positives, negatives, accuracy, precision, recall, area under the curve
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

# our model has the option for an label prediction bias, it's sequential, starts with an embedding layer, then bi-LSTM,
# a dropout layer follows for regularisation, and a dense final layer with softmax activation to output class probabilities
# we compile with the Adam optimizer at a low learning rate, use categorical cross-entropy as our loss function
def make_model(metrics = METRICS, output_bias=None):
    if output_bias is not None:        
        output_bias = tf.keras.initializers.Constant(output_bias)
    tok_input1 = keras.layers.Input(shape=(seq_length,), dtype='int32', name='tok_input1')
    red_input2 = keras.layers.Input(shape=(seq_length,), dtype='int32', name='red_input2')
    subchar_input3 = keras.layers.Input(shape=(seq_length,subchar_seq_length), dtype='int32', name='char_input3')

    emb_char = keras.layers.TimeDistributed(keras.layers.Embedding(output_dim=embed_size, input_dim=subchar_size, input_length=5,  mask_zero=True, trainable=True))(subchar_input3)
    char_enc = keras.layers.TimeDistributed(keras.layers.Bidirectional(keras.layers.LSTM(units=50, return_sequences=False, dropout=0.2)))(emb_char)
        

    x1 = keras.layers.Embedding(output_dim=embed_size, input_dim=vocab_size,  input_length=seq_length,  mask_zero=True, trainable=True)(tok_input1)
    x2 = keras.layers.Embedding(output_dim=embed_size, input_dim=red_size,  input_length=seq_length, mask_zero=True, trainable=True)(red_input2)
    x_cancat = keras.layers.concatenate([x1, x2, char_enc])
#     x_cancat = (x1)
    x_lstm = keras.layers.Bidirectional(keras.layers.LSTM(units=50, return_sequences=True, dropout=0.2))(x_cancat)
    x_drop = keras.layers.Dropout(0.5)(x_lstm)
    main_output = keras.layers.TimeDistributed(keras.layers.Dense(n_labs, activation='softmax', bias_initializer=output_bias))(x_drop)
    model = keras.models.Model(inputs=[tok_input1,red_input2, subchar_input3], outputs= main_output)
#     model = keras.models.Model(inputs=[tok_input1, pos_input2, char_input3], outputs= main_output)
    
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-3), loss=keras.losses.CategoricalCrossentropy(), metrics=metrics)
    return model


# Old lstm model
#     model = keras.Sequential([
#         keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=seq_length, mask_zero=True, trainable=True),
#         keras.layers.Bidirectional(keras.layers.LSTM(units=50, return_sequences=True, dropout=0.2)),  # 2 directions, 50 units each, concatenated (can change this)
#         keras.layers.Dropout(0.5),
#         keras.layers.TimeDistributed(keras.layers.Dense(n_labs, activation='softmax', bias_initializer=output_bias)),
#     ])






# early stopping criteria based on area under the curve: will stop if no improvement after 10 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', verbose=1, patience=10, mode='max', restore_best_weights=True)

# the number of training epochs we'll use, and the batch size (how many texts are input at once)
EPOCHS = 100
BATCH_SIZE = 128

print('**Defining a neural network**')
model = make_model()
model.summary()

True
**Defining a neural network**
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input3 (InputLayer)        [(None, 1019, 5)]    0                                            
__________________________________________________________________________________________________
tok_input1 (InputLayer)         [(None, 1019)]       0                                            
__________________________________________________________________________________________________
red_input2 (InputLayer)         [(None, 1019)]       0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 1019, 5, 128) 170240      char_input3[0][0]                
___________________________________________________________

In [ ]:
# %%time
# # evaluate our initial model
# results = model.evaluate(X, y, batch_size=BATCH_SIZE, verbose=0)
# print("Loss: {:0.4f}".format(results[0]))

In [ ]:
# figure out the label distribution in our fixed-length texts
from collections import Counter
all_labs = [l for lab in train_labs_padded for l in lab]
label_count = Counter(all_labs)
total_labs = len(all_labs)
print(label_count)
print(total_labs)

# use this to define an initial model bias
initial_bias=[(label_count[0]/total_labs), (label_count[1]/total_labs),
              (label_count[2]/total_labs), (label_count[3]/total_labs), (label_count[4]/total_labs)]
print('Initial bias:')
print(initial_bias)

Counter({4: 17589578, 0: 585226, 2: 585226, 3: 524721, 1: 131275})
19416026
Initial bias:
[0.03014138938627297, 0.006761167295511451, 0.03014138938627297, 0.027025149224666263, 0.9059309047072763]


In [ ]:
# prepare sequences and labels as numpy arrays, check dimensions
X = np.array(train_seqs_padded)
X_red = np.array(train_red_padded)
X_sub = np.array(train_subchar_seqs_padded)
y = np.array(train_labs_onehot)
print('Input sequence dimensions (n.docs, seq.length):')
print(X.shape)
print('Label dimensions (n.docs, seq.length, one-hot encoding of 4 NER labels):')
print(y.shape)

Input sequence dimensions (n.docs, seq.length):
(19054, 1019)
Label dimensions (n.docs, seq.length, one-hot encoding of 4 NER labels):
(19054, 1019, 5)


In [ ]:
# re-initiate model with bias
model = make_model(output_bias=initial_bias)
# and fit...
model.fit([X,X_red,X_sub], y, batch_size=32, epochs=10, callbacks = [early_stopping],  validation_split = 0.3)

Epoch 1/10


CancelledError: ignored

In [ ]:
X_test = np.array(test_seqs_padded)
X_test_rad = np.array(test_rad_padded)
# preds = np.argmax(model.predict(X_test), axis=-1)
preds = np.argmax(model.predict([X_test,X_test_rad]), axis=-1)
flat_preds = [p for pred in preds for p in pred]
print(Counter(flat_preds))

In [ ]:
len(preds)

In [ ]:
def postEditPred(pred):
    print(pred)
    ans = []
    for item in pred:
        if item == 0 or item == 3:
            ans.append(1)
        else:
            ans.append(0)
            
    assert len(ans) == len(pred)
    return ans

def splitSentence(st, pred):
    temp_st = st
    temp_pred = pred
    temp_pred[0] = 2
    buf = []
    result = []
    for (pre,char) in zip(temp_pred,temp_st):
        if(pre == 0 or pre == 3):
            result.append(buf)
            buf = []
        buf.append(char)
    if(len(buf) > 0):
        result.append(buf)
    return result
    
    



def sentencePrediction(dataset,prediction):
    data_temp = dataset.copy()
    assert len(data_temp) == len(prediction)
    raw_sents = data_temp["raws"]
    assert len(raw_sents) == len(prediction)
    ans = []
    for (st,pred) in zip(raw_sents,prediction):
        sptSt = splitSentence(st,pred)
        ans.append(sptSt)
    assert len(ans) == len(data_temp)
    data_temp["tokenList"]= ans
    return data_temp
    
def convertToPredSts(dataset):
    data_temp = dataset.copy()
    tokenList = data_temp["tokenList"]
    ans = []
    
    for item in tokenList:
        temp_st = list(map(lambda x : "".join(x), item))
        temp = "  ".join(temp_st)
        temp = temp + "  "
        ans.append(temp)
    return ans

In [ ]:
result_pred = sentencePrediction(test_feature,preds)
output_sts = convertToPredSts(result_pred)

f = open("./ans.txt", "w")
f.write("\n".join(output_sts))
f.close()